# 3. All about SPARQL

## 3.1. Finding out all about SPARQL

We will review the SPARQL documents, in particular the Query Language:

* <https://www.w3.org/TR/sparql11-query/>
    * also <https://www.w3.org/TR/sparql12-query/>
 
We will cover:

1. The multiple SPARQL specifications
2. Property Paths
3. Assignment
4. Aggregation, ORDER BY & LIMIT
5. GRAPH
6. Functions
    1. [SPARQL 1.1 Functions](https://www.w3.org/TR/sparql12-query/#SparqlOps)
7. DESCRIBE, CONSTRUCT & INSERT

## 3.1. The multiple SPARQL specifications

1. SPARQL Query Language - 1.1 & 1.2
2. SPARQL Update

## 3.2. Property Paths

```mermaid
flowchart LR
    d["Dataset A"]
    px["Person X"]
    d --prov:qualifiedAttribution--> qa1
    qa1 --prov:agent--> px    
    qa1 --prov:hadRole--> dr:custodian
```

There are two property paths here:

```mermaid
flowchart LR
    d --prov:qualifiedAttribution / prov:agent--> px
```

and

```mermaid
flowchart LR
    d --prov:qualifiedAttribution / prov:hadRole--> dr:custodian
```

and remember the person's name is a further step:

```mermaid
flowchart LR
    d["Dataset A"]
    px["Person X"]
    d --prov:qualifiedAttribution / prov:agent / schema:name--> px
```


In [29]:
from IPython.display import display, Markdown
from kurra.sparql import query
from kurra.utils import render_sparql_result


def table_print(r):
    display(Markdown(render_sparql_result(r)))
    

rdf_bn = """
PREFIX dr: <https://linked.data.gov.au/def/data-roles/>
PREFIX ex: <http://example.com/>
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

ex:a
    a schema:Dataset ;
    schema:name "Dataset A" ;
    prov:qualifiedAttribution [
        prov:agent people:px ;
        prov:hadRole dr:custodian ;
    ] ,
    [
        prov:agent people:py ;
        prov:hadRole dr:rightsHolder
    ] ;
.

people:px
    a schema:Person ;
    schema:name "Person X" ;
.

people:py
    a schema:Person ;
    schema:name "Person Y" ;
.
"""

q = """
PREFIX dr: <https://linked.data.gov.au/def/data-roles/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

SELECT ?name
WHERE {       
    ex:a prov:qualifiedAttribution/prov:agent/schema:name ?name .
}
"""

In [28]:
r = query(rdf_bn, q)
table_print(r)

| name |
| --- |
Person X
| Person Y |


## 3.2. Try it out

DESCRIBE:

```sparql
DESCRIBE <https://linked.data.gov.au/dataset/qld-addr/address/65cb1e52-fc1d-5dee-a2d2-ea7882d12c7e>

DESCRIBE <https://linked.data.gov.au/def/addr-part-types/countryName>

DESCRIBE <https://linked.data.gov.au/def/addr-part-types>
```

CONSTRUCT:
```sparql
PREFIX address: <https://linked.data.gov.au/dataset/qld-addr/address/>
PREFIX schema: <https://schema.org/>

CONSTRUCT {
  address:65cb1e52-fc1d-5dee-a2d2-ea7882d12c7e schema:comment "A nice place"
}
WHERE { GRAPH ?g {
  address:65cb1e52-fc1d-5dee-a2d2-ea7882d12c7e ?p ?o
} }
LIMIT 1
```